In [1]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import os

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import optuna
from optuna.trial import TrialState
from skimage import io

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Epochs = 20
classes = 10

In [3]:
# download dataset
# !wget -O dataset.tar.gz https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
# !wget -O labels.tar.gz https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz


In [4]:
# dataset class

class Cat_Dog_Dataset(Dataset):
    def __init__(self, label_dir, data_dir, train, transform=None):
        np.random.seed(0)
        self.label_dir = label_dir
        self.data_dir = data_dir
        self.transform = transform
        self.train = train
        self.filter_dataset()
        
    def class_vise_idx(self, labels, size):
        all_cats = labels[labels[:, 2] == '1']
        filtered_cats = all_cats[np.random.choice(len(all_cats), size=size, replace=False)]

        all_dogs = labels[labels[:, 2] == '2']
        filtered_dogs = all_dogs[np.random.choice(len(all_dogs), size=size, replace=False)]
        filtered_labels = np.vstack((filtered_cats, filtered_dogs))
        return np.delete(filtered_labels, [1, -1], axis=1)
        
    def filter_dataset(self):
        if self.train:
            label_path = os.path.join(self.label_dir, "trainval.txt")
            labels = np.loadtxt(label_path, dtype=object)
            self.labels = self.class_vise_idx(labels, size=750)

        else:
            label_path = os.path.join(self.label_dir, "test.txt")
            labels = np.loadtxt(label_path, dtype=object)
            self.labels = self.class_vise_idx(labels, size=250)
            
    def t_transform(self, label, n_classes=2):
        sample = np.eye(n_classes, dtype=int)
        if label == 1:
            return sample[0]
        if label == 2:
            return sample[1]
        
    def t(self, img):
        return self.transform(img)
        
    def __len__(self):
        return len(self.labels)
    
    def visual(self, img):
        plt.imshow(img)
        plt.show
    
    def __getitem__(self, idx):
        name, label = self.labels[idx]
        file_path = os.path.join(self.data_dir, name+'.jpg')
        img = np.array(Image.open(file_path).convert('RGB'))
        
        if self.transform:
            img = self.t(img)

        label = self.t_transform(int(label))
        return (img, label)

In [5]:
label_dir = "./data/annotations"
data_dir = "./data/images"

#have to apply more transforms 

t = transforms.Compose([transforms.ToTensor(),
#                         transforms.RandomResizedCrop(500),
                        transforms.ColorJitter(brightness=.5, hue=.3),
                        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
                        transforms.RandomPerspective(distortion_scale=0.6, p=0.5),
                        transforms.RandomVerticalFlip(p=0.5),
                        transforms.Resize((224, 224)),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                       ])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((224, 224)),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                       ])

train_dataset = Cat_Dog_Dataset(label_dir, data_dir, train=True, transform=t)
test_dataset = Cat_Dog_Dataset(label_dir, data_dir, train=False, transform=test_transform)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [12]:
def train(model, train_loader, optimizer, criterion, device):
    """
    Training for one epoch
    """
    loss_list = []
    for i, (images, labels) in enumerate(train_loader):
        # put on gpu
        images, labels = images.to(device), labels.type(torch.float32).to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        
        loss.backward()
        
        optimizer.step()
        
    avg_loss = np.mean(loss_list)
    return avg_loss, loss_list

@torch.no_grad()
def test(model, test_loader, criterion, device):
    """
    Testing the accuracy for validation set
    """
    correct = 0
    total = 0
    loss_list = []
    for images, labels in test_loader:
        images, labels = images.to(device), labels.type(torch.float32).to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        
        preds = torch.argmax(outputs, dim=1)
        gt = torch.argmax(labels, dim=1)
        correct += len(torch.where(preds==gt)[0] )
        total += len(labels)
    
    accuracy = correct / total * 100
    loss = np.mean(loss_list)
    
    return accuracy, loss
        
def training(model, optimizer, scheduler, criterion, train_loader, valid_loader, num_epochs):
    train_loss = []
    val_loss =  []
    loss_iters = []
    valid_acc = []
    
    for epoch in range(num_epochs):
        # validation epoch
        model.eval()  # important for dropout and batch norms
        accuracy, loss = test(
                    model=model, test_loader=valid_loader,
                    criterion=criterion, device=device
            )
        valid_acc.append(accuracy)
        val_loss.append(loss)
        
        # training epoch
        model.train()  # important for dropout and batch norms
        mean_loss, cur_loss_iters = train(model, train_loader, optimizer, criterion, device)
        scheduler.step()
        train_loss.append(mean_loss)
        loss_iters = loss_iters + cur_loss_iters
        
        if(epoch % 5 == 0 or epoch==num_epochs-1):
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"    Train loss: {round(mean_loss, 5)}")
            print(f"    Valid loss: {round(loss, 5)}")
            print(f"    Accuracy: {accuracy}%")
            print("\n")
    
    print(f"Training completed")
    return train_loss, val_loss, loss_iters, valid_acc

In [13]:
# get pretrained models
vgg = models.vgg16(pretrained=True)
resnet = models.resnet18(pretrained=True)
dense_net = models.densenet161(pretrained=True)

In [14]:
resnet.fc = nn.Sequential(
                    nn.Linear(512, 256),
                    nn.ReLU(),
                    nn.Linear(256, 2))

In [15]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(resnet.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [16]:
train_loss, val_loss, loss_iters, valid_acc = training(model=resnet.to(device), 
                                                      optimizer=optimizer,
                                                      scheduler=scheduler,
                                                      criterion=criterion,
                                                      train_loader=train_loader,
                                                      valid_loader=test_loader,
                                                      num_epochs=15)


Epoch 1/15
    Train loss: 0.3968
    Valid loss: 0.66867
    Accuracy: 61.8%


Epoch 6/15
    Train loss: 0.1613
    Valid loss: 0.09965
    Accuracy: 96.2%


Epoch 11/15
    Train loss: 0.05584
    Valid loss: 0.06756
    Accuracy: 97.0%


Epoch 15/15
    Train loss: 0.04709
    Valid loss: 0.05879
    Accuracy: 97.39999999999999%


Training completed
